# Section 8.2: Time Matrix Multiply

Adapted from: "Guide to Fortran 2008 Programming" by Walter S. Brainerd (Springer 2015)

## Program to demonstrate the performance improvement of using matmul over do loops.

```fortran
program time_matrix_multiply

    ! Compare times of the matmul intrinsic cs DO loops

    implicit none
    integer, parameter :: n = 10
    real, dimension (n, n) :: a, b, c1, c2
    character(len=8) :: date
    real :: start_time_1, stop_time_1, start_time_2, stop_time_2
    real :: total_time_1, total_time_2
    integer :: i, j, k
    character(len=*), parameter :: form = "(t2, a, f0.10, a)"

    ! Get date to print on report

    call date_and_time(date = date)

    print *, "Timing report dated: " // date(1:4) &
        // "-" // date(5:6) // "-" // date(7:8)

    call random_seed()
    call random_number(a)
    call random_number(b)
    call cpu_time(start_time_1)
   
    ! Lines below added for effect
    ! The matrices of random values will be printed to the screen
    write (*, "(a)") "Matrix A"
    write (*, "(10f10.3)") (a(i,:), i=1,n)

    print *
    write (*, "(a)") "Matrix B"
    write (*, "(10f10.3)") (b(i,:), i=1,n)

    c1 = 0

    do k = 1, n
        do j = 1, n
            do i = 1, n
                c1(i, j) = c1(i, j) + a(i, k) * b(k, j)
            end do
        end do
    end do

    call cpu_time(stop_time_1)

    total_time_1 = stop_time_1 - start_time_1

    print *

    write (*, "(a)") "Matrix C1 is AxB using loops."
    write (*, "(10f10.3)") (c1(i,:), i=1,n)

    call cpu_time(start_time_2)
    c2 = matmul(a, b)
    call cpu_time(stop_time_2)

    total_time_2 = stop_time_2 - start_time_2

    print *

    write (*, "(a)") "Matrix C2 is AxB using matmul."
    write (*, "(10f10.3)") (c1(i,:), i=1,n)

    print *

    write (*, form) "Time of Do loop version is: ", total_time_1, " seconds."
    write (*, form) "Time of matmul version is: ", total_time_2, " seconds."

    print *

    if (any(abs(c1-c2) > 1.0e-4)) then
        write (*,  "(a)") "There are significantly different values between the matrices."
    else
        write (*,  "(a)") "The results are approximately the same."
    end if

    print *
    write (*, "(a, f10.3, a)") "The speedup ratio is: ", total_time_1/total_time_2, "x"

end program time_matrix_multiply
```

The above program is compiled and run using Fortran Package Manager (fpm):

In [1]:
import os
root_dir = os.getcwd()

In [2]:
code_dir = root_dir + "/" + "Fortran_Code/Section_8_2_Time_Matrix_Multiply"

In [3]:
os.chdir(code_dir)

In [4]:
build_status = os.system("fpm build 2>/dev/null")

[  0%] section_8_2_time_matrix_multip
[ 50%] section_8_2_time_matrix_multip  done.
[ 50%] Section_8_2_Time_Matrix_Multip
[100%] Section_8_2_Time_Matrix_Multip  done.
[100%] Project compiled successfully.


In [5]:
exec_status = os.system("fpm run 2>/dev/null")

 Timing report dated: 2023-12-26
Matrix A
     0.392     0.117     0.102     0.897     0.521     0.101     0.492     0.932     0.118     0.910
     0.329     0.923     0.698     0.287     0.777     0.093     0.733     0.691     0.867     0.992
     0.788     0.716     0.120     0.019     0.050     0.479     0.580     0.977     0.179     0.250
     0.719     0.275     0.871     0.549     0.699     0.472     0.858     0.487     0.374     0.520
     0.240     0.211     0.135     0.824     0.307     0.755     0.145     0.042     0.556     0.343
     0.893     0.653     0.760     0.929     0.974     0.672     0.407     0.253     0.163     0.588
     0.157     0.384     0.564     0.180     0.478     0.752     0.161     0.125     0.352     0.089
     0.356     0.484     0.082     0.016     0.686     0.887     0.466     0.694     0.060     0.727
     0.931     0.617     0.548     0.572     0.953     0.511     0.813     0.073     0.536     0.533
     0.822     0.079     0.860     0.944     0.10